In [1]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from qiskit import BasicAer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.datasets import wine
from qiskit_machine_learning.circuit.library import RawFeatureVector

seed = 1376
algorithm_globals.random_seed = seed

# Use Wine data set for training and test data
feature_dim = 4  # dimension of each data point
training_size = 12
test_size = 4

# training features, training labels, test features, test labels as np.array,
# one hot encoding for labels
training_features, training_labels, test_features, test_labels = \
    wine(training_size=training_size, test_size=test_size, n=feature_dim)

In [3]:
feature_map = RawFeatureVector(feature_dimension=feature_dim)
ansatz = TwoLocal(feature_map.num_qubits, ['ry', 'rz'], 'cz', reps=3)

In [4]:
#from qiskit.circuit.library import ZZFeatureMap, TwoLocal

#adhoc_feature_map = ZZFeatureMap(feature_dimension=2, reps=2)
#adhoc_var_form = TwoLocal(2, ['ry', 'rz'], 'cz', reps=2)

adhoc_circuit = feature_map.compose(ansatz)
adhoc_circuit.measure_all()
adhoc_circuit.decompose().draw()

Traceback (most recent call last):
  Input In [4] in <cell line: 8>
    adhoc_circuit.decompose().draw()
  File /opt/conda/lib/python3.8/site-packages/qiskit/circuit/library/blueprintcircuit.py:103 in decompose
    return super().decompose(gates_to_decompose)
  File /opt/conda/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:1530 in decompose
    decomposed_dag = pass_.run(circuit_to_dag(self))
  File /opt/conda/lib/python3.8/site-packages/qiskit/transpiler/passes/basis/decompose.py:90 in run
    if node.op.definition is None:
  File /opt/conda/lib/python3.8/site-packages/qiskit/circuit/instruction.py:232 in definition
    self._define()
  File /opt/conda/lib/python3.8/site-packages/qiskit_machine_learning/circuit/library/raw_feature_vector.py:170 in _define
    raise QiskitError("Cannot define a ParameterizedInitialize with unbound parameters")
QiskitError: 'Cannot define a ParameterizedInitialize with unbound parameters'

Use %tb to get the full traceback.


In [2]:
vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          optimizer=COBYLA(maxiter=100),
          quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                           shots=1024,
                                           seed_simulator=seed,
                                           seed_transpiler=seed)
          )
vqc.fit(training_features, training_labels)

score = vqc.score(test_features, test_labels)
print(f"Testing accuracy: {score:0.2f}")

Cannot compute gradient operator! Continuing without gradients!


Testing accuracy: 1.00


In [4]:
print(test_labels)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]
